In [1]:
source("helper.R")
library(lubridate)
library(matrixStats)
max_expid<-get_max("Log")+1
dataset_name="SPY"

family_name="stochvol.t"
model_name="matern"
SAMPLE_SIZE=100000
exp_name<-glue("Pricing_{dataset_name}_Model_{family_name}_{model_name}")
log_dir<-glue("Log/{max_expid}.{exp_name}")
logger=get_logger(log_dir, log_name = "log", debug = TRUE)
futile.logger::flog.info("start experiment", name = "log")
futile.logger::flog.info(glue("number of simulations: {SAMPLE_SIZE}"), name = "log")
## get logger

seed_random<-123
n=30

if (dataset_name=="China"){
risk_free_rate<-0.02
}else if (dataset_name=="Saudi"){
risk_free_rate<-0.02
}else if (dataset_name=="SPY"){
risk_free_rate<-0.02
}



futile.logger::flog.info(glue("risk-free rate: {risk_free_rate}"), name = "log")
set.seed(seed_random)

futile.logger::flog.info(glue("use seed: {seed_random}"), name = "log")

futile.logger::flog.info(glue("number of experiments: {n}"), name = "log")
if (dataset_name=="China"){
dataset <- load_data("data/IFclose.csv")
option_data <- read.csv("data/IFtest.csv")


}else if (dataset_name=="Saudi"){
dataset <- load_data("data/1180_vol.csv")

}else if (dataset_name=="SPY"){

dataset <- load_data("data/SPY.csv") 
option_data <- read.csv("data/SPY_testPeriod.csv")

}else{

    stop("Error: Invalid data set")
}




Warning message:
“package ‘INLA’ was built under R version 4.5.0”
Loading required package: Matrix



This is INLA_25.06.07 built 2025-06-11 18:54:45 UTC.
 - See www.r-inla.org/contact-us for how to get help.
 - List available models/likelihoods/etc with inla.list.models()
 - Use inla.doc(<NAME>) to access documentation


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


This is rSPDE 2.5.1
- See https://davidbolin.github.io/rSPDE for vignettes and manuals.


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count




INFO [2025-09-11 11:02:33] start experiment
INFO [2025-09-11 11:02:33] number of simulations: 1e+05
INFO [2025-09-11 11:02:33] risk-free rate: 0.02
INFO [2025-09-11 11:02:33] use seed: 123
INFO [2025-09-11 11:02:33] number of experiments: 30


Rows: 3687 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): act_symbol
dbl  (5): open, high, low, close, volume
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [2]:
test_day_start=option_data$date[1]
test_day_end=option_data$date[nrow(option_data)]
best.rspde.order<-2
best.prior.range.nominal<-137.3123
best.prior.std.dev.nominal<-0.5628


In [ ]:
results <- data.frame(
  date = as.Date(character()),
  Call_put = character(),
  strike = numeric(),
  option_theoretical_price = numeric(),
  expiration = as.Date(character())
)

for (rolling_test_day_start in sort(unique(option_data$date))){

    futile.logger::flog.info(glue("rolling_test_day_start: {rolling_test_day_start}"), name = "log")
    ## Prepare data and mesh
    subset_data <- dataset[dataset$date <= as.Date(rolling_test_day_start), ]
    option_data_subset <- option_data[option_data$date == rolling_test_day_start, ]
    max(option_data_subset$expiration)
    days_to_expiration=as.numeric(as.Date(max(option_data_subset$expiration))-as.Date(rolling_test_day_start))
    
    index_start=subset_data$time[length(subset_data$time)]

    last_price<-subset_data$close[length(subset_data$close)]
    all_days <- seq.Date(from = as.Date(rolling_test_day_start) + 1, by = "day", length.out = days_to_expiration)
    biz_days <- all_days[!weekdays(all_days) %in% c("Saturday", "Sunday")]
    index<-seq_len(days_to_expiration)[!weekdays(all_days) %in% c("Saturday", "Sunday")]

    new_data <- data.frame(
    logreturn = NA,
    times = max(dataset$times) + index,
    date = biz_days,
    close = NA
    )
    subset_data_expanded <- rbind(dataset, new_data)

    ## Build model
    if (model_name=="matern"){
    latent_model <- rspde.matern(
        mesh = fm_mesh_1d(subset_data_expanded$times),
        parameterization = "matern",
        rspde.order = best.rspde.order,
        prior.range.nominal = best.prior.range.nominal,
        prior.std.dev.nominal = best.prior.std.dev.nominal
    )
        
        formula_latent <- logreturn ~ -1 + Intercept(1)+ field(times, model = latent_model)
    }
        
    fit<-bru(formula_latent, family=family_name, data = subset_data_expanded)
    result_fit<- rspde.result(fit, "field", latent_model, parameterization="matern")

    futile.logger::flog.info(glue("result_fit: {result_fit}"), name = "log")
    summary_df <- as.data.frame(summary(result_fit))
    flog.info(
    paste0("\n", paste(capture.output(print(summary_df)), collapse = "\n")),
    name = "log"
    )

    ## Generate samples
    new_data <- subset_data_expanded[is.na(subset_data_expanded$logreturn), ]
    discount_factor<- (1+risk_free_rate)**(-dim(new_data)[1]/252)
    delta_S_T<-generate(fit, new_data, ~ exp((field+Intercept)/2), n.samples = SAMPLE_SIZE)

    if(family_name=="stochvol.t"){
    dof <- fit$summary.hyperpar$mean[1]
    random_return_rate<-simulate_T(dof,delta_S_T)
    }else if (family_name=="stochvol"){
    random_return_rate<-matrix(
            rnorm(length(delta_S_T),0,1),
            nrow = nrow(delta_S_T),
            ncol = ncol(delta_S_T)
        )
    }
    
    futile.logger::flog.info(glue("generate samples"), name = "log")

    ## calculate option price

    cumulative_random_return_rate <- colCumsums(random_return_rate)
    expire_all <-sort(unique(option_data_subset$expiration))
    for (x in expire_all){
        number_of_days<-as.numeric(as.Date(x) - as.Date(rolling_test_day_start))
        index_in_matrix<-which(unlist(index) == number_of_days)

        return_rate_to_end<-exp(cumulative_random_return_rate[index_in_matrix,])
        simulated_final_prices <- last_price* return_rate_to_end
        ### call option
        call_option_data<-option_data_subset[(option_data_subset$call_put=="Call") & (option_data_subset$expiration==x),]
        
        

        for (i in 1:length(call_option_data$strike)) {
        strike <- call_option_data$strike[i]
        theoretical_price_call <- expected_payoff_for_European_option(
            simulated_final_prices, K = strike, type = "call", discount_factor = discount_factor
        )
        results <- rbind(results, data.frame(
            date = as.Date(rolling_test_day_start),
            Call_put = "Call",
            strike = strike,
            option_theoretical_price = theoretical_price_call,
            expiration = x
        ))
        }
        
        #put option
        put_option_data<-option_data_subset[(option_data_subset$call_put=="Put") & (option_data_subset$expiration==x),]


        for (i in 1:length(put_option_data$strike)) {
        strike <- put_option_data$strike[i]
        theoretical_price_put <- expected_payoff_for_European_option(
            simulated_final_prices, K = strike, type = "put", discount_factor = discount_factor
        )
        results <- rbind(results, data.frame(
            date = as.Date(rolling_test_day_start),
            Call_put = "Put",
            strike = strike,
            option_theoretical_price = theoretical_price_put,
            expiration = x
        ))
        }
    }
}


In [ ]:

write.csv(results, "results.csv",row.names = FALSE)